# Preprocessing

## Libraries

In [2]:
import pandas as pd
import holidays

## Datapreprocessing

In [3]:

def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format="%m/%d/%Y %H:%M")
    except ValueError:
        # If that fails, try the ISO8601 format: year-month-day hour:minute:second
        try:
            return pd.to_datetime(date_str, format="%Y-%m-%d %H:%M:%S")
        except ValueError:
            # Return NaT if both formats fail
            print(f"Could not parse date: {date_str}")
            return pd.NaT


# Load the CSV file into a pandas dataframe
df = pd.read_csv('ColoradoData.csv')

# Strip extra whitespace from the column names
df.columns = [col.strip() for col in df.columns]

# Strip extra whitespace from the date columns
df['Start_Date___Time'] = df['Start_Date___Time'].str.strip()
df['End_Date___Time'] = df['End_Date___Time'].str.strip()

# Now parse the datetime columns using the parse_date function
df['Start_DateTime'] = df['Start_Date___Time'].apply(parse_date)
df['End_DateTime'] = df['End_Date___Time'].apply(parse_date)

# Convert duration columns to timedelta objects
df['Total_Duration'] = pd.to_timedelta(df['Total_Duration__hh_mm_ss_'])
df['Charging_Time'] = pd.to_timedelta(df['Charging_Time__hh_mm_ss_'])

# Convert Energy consumption to a numeric type (handle errors)
df['Energy_Consumption'] = pd.to_numeric(df['Energy__kWh_'], errors='coerce')

# Remove the columns that are no longer needed
df = df.drop(columns=['Start_Date___Time', 'End_Date___Time', 'Total_Duration__hh_mm_ss_',
             'Charging_Time__hh_mm_ss_', 'Energy__kWh_', 'ObjectID', 'ObjectId2', 'Start_Time_Zone', 'End_Time_Zone', 'Port_Type', 'GHG_Savings__kg_', 'Gasoline_Savings__gallons_', 'Zip_Postal_Code', 'City'])

df.set_index('Start_DateTime', inplace=True)

## More data cleaning
# Remove rows with negative energy consumption
df = df[df['Energy_Consumption'] >= 0]

# Remove rows with missing values
df = df.dropna()

# Remove rows with zero charging time
df = df[df['Charging_Time'] > pd.Timedelta(0)]

# Remove rows with zero energy consumption
df = df[df['Energy_Consumption'] > 0]

# Remove rows with zero total duration
df = df[df['Total_Duration'] > pd.Timedelta(0)]

# Remove rows with total duration less than charging time
df = df[df['Total_Duration'] >= df['Charging_Time']]

# Verify the conversion by printing the data types
print("Optimized dataset:")
print(df.dtypes)

print("Optimized dataset:")
print(df)

df = df.sort_values(by=['Start_DateTime'], inplace=True)

# Save the cleaned data to a new CSV file
df.to_csv('CleanedColoradoData.csv')

Optimized dataset:
Station_Name                   object
Address                        object
State_Province                 object
End_DateTime           datetime64[ns]
Total_Duration        timedelta64[ns]
Charging_Time         timedelta64[ns]
Energy_Consumption            float64
dtype: object
Optimized dataset:
                                        Station_Name             Address  \
Start_DateTime                                                             
2018-01-01 17:49:00  BOULDER / JUNCTION ST1           2280 Junction Pl     
2018-01-02 08:52:00  BOULDER / JUNCTION ST1           2280 Junction Pl     
2018-01-02 21:11:00  BOULDER / JUNCTION ST1           2280 Junction Pl     
2018-01-03 09:19:00  BOULDER / ALPINE ST1             1275 Alpine Ave      
2018-01-03 14:13:00  BOULDER / BASELINE ST1           900 Baseline Rd      
...                                              ...                 ...   
2023-11-30 19:11:00  MUNICIPAL SC / 1100WALNUT1       1100 Walnut         

AttributeError: 'NoneType' object has no attribute 'to_csv'

## Aggregating (Day, Charging Station, Addresses) 

In [14]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('cleaned_ColoradoData2.csv')

# Convert date/time columns to datetime
df['Start_DateTime'] = pd.to_datetime(df['Start_DateTime'])
df['End_DateTime'] = pd.to_datetime(df['End_DateTime'])

# Set Start_DateTime as the index
df.set_index('Start_DateTime', inplace=True)

# Convert Energy_Consumption to numeric (if not already)
df['Energy_Consumption'] = pd.to_numeric(df['Energy_Consumption'])

# Convert Charging_Time and Total_Duration from string to timedelta
df['Charging_Time'] = pd.to_timedelta(df['Charging_Time'])
df['Total_Duration'] = pd.to_timedelta(df['Total_Duration'])

df['Charging_Time_sec'] = df['Charging_Time'].dt.total_seconds()
df['Total_Duration_sec'] = df['Total_Duration'].dt.total_seconds()


# Resample the data by day to aggregate energy consumption, charging time, and total duration
daily_df = df.resample('D').agg({
    'Energy_Consumption': 'sum',
    'Charging_Time_sec': 'mean',
    'Total_Duration_sec': 'mean',
    'Station_Name': 'count'  # Counting sessions per day
}).rename(columns={'Station_Name': 'daily_sessions'})

# Count daily sessions using resample size (this counts the number of rows per day)
daily_sessions = df.resample('D').size()
daily_df['daily_sessions'] = daily_sessions

# Rename the columns to match your desired output
daily_df.rename(columns={
    'Energy_Consumption': 'aggregated_energy_consumption',
    'Charging_Time': 'aggregated_average_charging_time',
    'Total_Duration': 'aggregated_average_total_duration'
}, inplace=True)

daily_df['avg_charging_time_hours'] = daily_df['Charging_Time_sec'] / 3600
daily_df['avg_total_duration_hours'] = daily_df['Total_Duration_sec'] / 3600

# Drop the columns that are no longer needed
daily_df = daily_df.drop(columns=['Charging_Time_sec', 'Total_Duration_sec'])

# Reset the index to have a 'date' column (renaming the index column from 'Start_DateTime' to 'date')
daily_df = daily_df.reset_index().rename(columns={'Start_DateTime': 'date'})

# Display the resulting DataFrame
print(daily_df.columns)

daily_df = daily_df.drop(columns=['level_0', 'index'], errors='ignore')

# Save the aggregated data to a new CSV file

Index(['date', 'aggregated_energy_consumption', 'daily_sessions',
       'avg_charging_time_hours', 'avg_total_duration_hours'],
      dtype='object')


## Features

In [16]:
# Energy pr session
import numpy as np


daily_df['avg_energy_per_session'] = daily_df['aggregated_energy_consumption']/daily_df['daily_sessions']

# Charging Utilization Ratio (CUR)
daily_df['CUR'] = daily_df['avg_charging_time_hours'] / daily_df['avg_total_duration_hours']

# Total Duration Utilization Ratio (TDUR)
daily_df['TDUR'] = daily_df['avg_total_duration_hours'] / 24  # 24 hours in a day

# Daily Change in Energy Consumption
daily_df['daily_change_in_energy_consumption'] = daily_df['aggregated_energy_consumption'].diff()

# Rolling Average of Energy Consumption
daily_df['rolling_avg_energy_consumption_1d'] = daily_df['aggregated_energy_consumption'].rolling(window=1).mean()
daily_df['rolling_sessions_1d'] = daily_df['daily_sessions'].rolling(window=1).mean()

# Day of the week and Weekend indicator
daily_df['day_of_week'] = daily_df['date'].dt.dayofweek # Monday=0, Sunday=6
daily_df['is_weekend'] = daily_df['day_of_week'].isin([5, 6]).astype(int)

# Public holidays
us_holidays = holidays.US()
daily_df['is_public_holiday'] = daily_df['date'].dt.date.astype(str).map(us_holidays.get).notnull().astype(int)

# Data features
daily_df['Day'] = daily_df['date'].dt.day
daily_df['Hour'] = daily_df['date'].dt.hour

# Cos, Sin transformation of day of the year
daily_df['HourSin'] = np.sin(2 * np.pi * daily_df['Hour'] / 24)
daily_df['HourCos'] = np.cos(2 * np.pi * daily_df['Hour'] / 24)

print(daily_df.columns)

# Save the normalized data to a new CSV file
daily_df.to_csv('features_ColoradoData.csv', index=False)

Index(['date', 'aggregated_energy_consumption', 'daily_sessions',
       'avg_charging_time_hours', 'avg_total_duration_hours',
       'avg_energy_per_session', 'CUR', 'TDUR',
       'daily_change_in_energy_consumption', 'rolling_avg_energy_consumption',
       'rolling_sessions_7d', 'day_of_week', 'is_weekend', 'is_public_holiday',
       'rolling_avg_energy_consumption_1d', 'rolling_sessions_1d', 'Day',
       'Hour', 'HourSin', 'HourCos'],
      dtype='object')
